In [4]:
realinput = '653427918'
testinput = '389125467'

In [5]:
import numpy as np

# Part 1

In [255]:
cups = np.array([int(v) for v in realinput])

In [256]:
indices = np.arange(len(cups))

In [257]:
debug = False

In [258]:
Ntake = 3

In [259]:
for move in range(1, 100+1):
    if debug: 
        print(f'-- move {move} --')
        print('cups:    ', cups)
        print('indices: ', indices)

    lowest_cup = min(cups)
    highest_cup = max(cups)

    [currentcup_i], = np.nonzero(indices == 0)
    currentcup_v = cups[currentcup_i]

    selected = (1 <= indices) & (indices <= Ntake)
    pick_up = cups[selected][np.argsort(indices[selected])]
    cups = cups[~selected]

    if debug: print('pick up: ', pick_up)

    destination_v = currentcup_v

    while True:
        destination_v -= 1
        if destination_v < lowest_cup:
            destination_v = highest_cup
        if destination_v in cups:
            [destination_index], = np.nonzero(cups == destination_v)
            destination_index += 1
            break

    if debug: print('destination:', destination_index)

    cups = np.concatenate((cups[:destination_index], pick_up, cups[destination_index:]))
    [currentcup_i], = np.nonzero(cups == currentcup_v)
    indices = (np.arange(len(cups)) - currentcup_i - 1) % len(cups)
    
    if debug: print()

In [260]:
[one_i], = np.nonzero(cups == 1)
''.join(str(n) for n in list(cups[one_i+1:]) + list(cups[:one_i]))

'76952348'

# Part 2 unoptimized

In [21]:
Nvalues = 1_000_000

In [22]:
Nmoves = 10

In [23]:
cups = np.array([int(v) for v in realinput])

In [24]:
highest_cup = max(cups)

In [25]:
cups = np.concatenate((cups, np.arange(highest_cup+1, Nvalues + 1)))

In [26]:
# @numba.jit
# def play(cups, Nmoves):
#     cups = cups.copy()
indices = np.arange(len(cups))

debug = True

Ntake = 3

lowest_cup = min(cups)
highest_cup = max(cups)

for move in range(1, Nmoves+1):
    if debug: 
        print(f'-- move {move} --')
        print('cups:    ', cups[:30])
        print('indices: ', indices[:30])

    [currentcup_i], = np.nonzero(indices == 0)
    currentcup_v = cups[currentcup_i]

    selected = (1 <= indices) & (indices <= Ntake)
    pick_up = cups[selected][np.argsort(indices[selected])]
    cups = cups[~selected]

    if debug: print('pick up: ', pick_up)

    destination_v = currentcup_v

    while True:
        destination_v -= 1
        if destination_v < lowest_cup:
            destination_v = highest_cup
        if (destination_v == cups).any():
            [destination_index], = np.nonzero(cups == destination_v)
            destination_index += 1
            break
    print('destination:', destination_v)
            
    cups = np.concatenate((cups[:destination_index], pick_up, cups[destination_index:]))
    [currentcup_i], = np.nonzero(cups == currentcup_v)
    indices = (np.arange(len(cups)) - currentcup_i - 1) % len(cups)
#     return cups

-- move 1 --
cups:     [ 6  5  3  4  2  7  9  1  8 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30]
indices:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
pick up:  [5 3 4]
destination: 2
-- move 2 --
cups:     [ 6  2  5  3  4  7  9  1  8 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30]
indices:  [999999      0      1      2      3      4      5      6      7      8
      9     10     11     12     13     14     15     16     17     18
     19     20     21     22     23     24     25     26     27     28]
pick up:  [5 3 4]
destination: 1
-- move 3 --
cups:     [ 6  2  7  9  1  5  3  4  8 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30]
indices:  [999998 999999      0      1      2      3      4      5      6      7
      8      9     10     11     12     13     14     15     16     17
     18     19     20     21     22     23     24     25     26     27]
pick up:  [9 1 5]
destination: 6
--

In [ ]:
# %%time
cups = play(cups, Nmoves)

In [ ]:
[one_i], = np.nonzero(cups == 1)

In [ ]:
np.prod(cups.take(one_i + np.arange(2) + 1, mode='wrap'))

# Part 2

In [273]:
Nvalues = 1_000_000

In [274]:
cups = np.array([int(v) for v in realinput])

In [275]:
short_locations = np.zeros(len(cups) + 1).astype(int)

In [276]:
short_locations[cups] = np.arange(len(cups))

In [277]:
locations = np.arange(Nvalues+2) - 1

In [278]:
locations[:len(cups)+1] = short_locations

In [279]:
locations = locations.tolist()

In [280]:
cups = np.concatenate((cups, np.arange(highest_cup+1, Nvalues + 1)))

In [281]:
lowest_cup = min(cups)
highest_cup = max(cups)

In [282]:
Nmoves = 10_000_000

In [283]:
from tqdm.auto import tqdm

In [284]:
import numba

In [285]:
debug = False

In [286]:
# @numba.jit
# def play(cups, Nmoves):
Ntake = 3
Ncups = len(cups)
take = np.arange(3)
cups = cups.copy()
currentcup_i = 0

lowest_cup = min(cups)
highest_cup = max(cups)

for move in tqdm(range(1, Nmoves+1)):
    if debug: 
        print(f'-- move {move} --')
        print('cups:    ', ' '.join(f'({v})' if i == currentcup_i else f'{v}' for i, v in enumerate(cups[:30])))

    currentcup_v = cups[currentcup_i]

    pickup_i = (currentcup_i + take + 1) % Ncups
    pick_up = cups[pickup_i]

    if debug: print('pick up: ', pick_up)
    

    
    destination_v = currentcup_v

    while True:
        destination_v -= 1
        if destination_v < lowest_cup:
            destination_v = highest_cup+1
            continue
        if (destination_v == pick_up).any():
            continue
        break

    if debug: print('destination_v:', destination_v)
        
    buffer = []
    positions = pickup_i.tolist()

    destination_i = (currentcup_i + Ntake + 1) % Ncupsc
    while True:
        cup = cups[destination_i]
        positions.append(destination_i)
        buffer.append(cup)
        if cup == destination_v:
            break
        destination_i = (destination_i + 1) % Ncups

    buffer += pick_up.tolist()

    if debug:
        print('buffer:', buffer)
        print('positions:', positions)

    cups[positions] = buffer
    currentcup_i = (currentcup_i + 1) % Ncups

# return cups

KeyboardInterrupt: 

In [271]:
%%time
cups = play(cups, Nmoves)

NameError: name 'play' is not defined

In [272]:
[one_i], = np.nonzero(cups == 1)
''.join(str(n) for n in list(cups[one_i+1:]) + list(cups[:one_i]))

'69428375'

# Part 2
Completely different data structure - custom linked list

In [484]:
Nvalues = 1_000_000
# Nvalues = 9

In [485]:
class Cup:
    def __init__(self, value):
        self.value = value
        self.next = None
        
    def __repr__(self):
        return str(self.value)

In [486]:
cups_by_value = {}

In [487]:
numbers = list(range(1, Nvalues + 1))

In [488]:
for i, v in enumerate(realinput):
    numbers[i] = int(v)

In [489]:
numbers[:10]

[6, 5, 3, 4, 2, 7, 9, 1, 8, 10]

In [490]:
previous_cup = None
for number in numbers:
    cup = Cup(number)
    cups_by_value[number] = cup
    if previous_cup:
        previous_cup.next = cup
    previous_cup = cup
cup.next = current_cup = cups_by_value[numbers[0]]

In [491]:
Nmoves = 10_000_000

In [492]:
debug = False

In [493]:
Ntake = 3

for move in tqdm(range(1, Nmoves+1)):
    pickup = []
    pickup_set = set()
    
    
    if debug:
        print(f'-- move {move} --')
        print('cups:', end='')

        printcup = current_cup
        while printcup.next != current_cup:
            print(printcup, end=' ')
            printcup = printcup.next
        print(printcup, end=' ')

        print()

    
    # The crab picks up the three cups that are immediately 
    # clockwise of the current cup.  
    pickup_cup = current_cup.next
    for _ in range(Ntake):
        pickup.append(pickup_cup)
        pickup_set.add(pickup_cup.value)
        pickup_cup = pickup_cup.next

    # They are removed from the circle;
    # cup spacing is adjusted as necessary to maintain the circle.
    current_cup.next = pickup_cup
    
    destination_v = current_cup.value

    if debug: print('pick up:', pickup)
        
    while True:
        destination_v -= 1
        if destination_v < 1:
            destination_v = Nvalues+1
            continue
        if destination_v in pickup_set:
            continue
        break

    if debug: print('destination_v:', destination_v)
        
    destination_cup = cups_by_value[destination_v]
    
    # The crab places the cups it just picked up so that they are immediately clockwise of the destination cup. They keep the same order as when they were picked up.
    endpoint = destination_cup.next
    destination_cup.next = pickup[0]
    pickup[-1].next = endpoint
    
    # The crab selects a new current cup: the cup which is immediately clockwise of the current cup.
    current_cup = current_cup.next
        
    if debug:
        print()
    

In [494]:
cup1 = cups_by_value[1]

In [495]:
cupvalues = []
cup = cup1
for _ in range(2):
    cup = cup.next
    cupvalues.append(cup.value)

In [496]:
cupvalues

[864446, 84184]

In [497]:
cupvalues[0]*cupvalues[1]

72772522064